In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.sparse.linalg import eigsh
#from pretrainedModel import pretrainedModel
from tensorflow import keras
from PIL import Image
from sklearn.preprocessing import StandardScaler
import torch
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
import time
import warnings
from sklearn.cluster import KMeans
import sys
import os

from copy import deepcopy

In [2]:
import random
import torch.nn as nn
import numpy as np
import torch
import math
import torch.optim as optim
from torch.utils.data import DataLoader

root = '../'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def get_dataloader(X_train, y_train, one_hot=True, batchSize = 100):
    y_train = torch.tensor(y_train, dtype=torch.long, device=device)

    if one_hot:
        y_train = torch.nn.functional.one_hot(y_train)

    train_data = []
    for i in range(len(X_train)):
        train_data.append([X_train[i], y_train[i]])
    dataloader = DataLoader(train_data, batch_size=batchSize, shuffle=True)


    return dataloader

In [5]:
class DTSHLoss(torch.nn.Module):
    def __init__(self):
        super(DTSHLoss, self).__init__()

    def forward(self, u, y, LAMBDA=1, ALPHA=1):
        #LAMBDA = 1
        #ALPHA  = 1

        inner_product = u @ u.t()   # Similarity Matrix
        s = y @ y.t() > 0           # A matrix that show if the two idexes are the same or not
        count = 0

        loss1 = 0
        for row in range(s.shape[0]):
            # if has positive pairs and negative pairs
            if s[row].sum() != 0 and (~s[row]).sum() != 0:
                count += 1
                theta_positive = inner_product[row][s[row] == 1]                
                theta_negative = inner_product[row][s[row] == 0]

                triple = (theta_positive.unsqueeze(1) - theta_negative.unsqueeze(0) - ALPHA ).clamp(min=-100,max=50)
                loss1 += -(triple - torch.log(1 + torch.exp(triple))).mean()

        if count != 0:
            loss1 = loss1 / count
        else:
            loss1 = 0

        loss2 = LAMBDA * (u - u.sign()).pow(2).mean()

        return loss1 + loss2

In [ ]:
def earlyStop(LossList, n = 10):
    bestVal = min(LossList)

    bestVal_i = LossList.index(bestVal)

    if bestVal_i < len(LossList) - n: return True



def HPO(HP, X_train, y_train, do_one_hot=True, earlyStop_num=20, timedStop=60):
    time_start = time.time()

    data = {}

    LAMBDA= HP["lambda"] 
    ALPHA=  HP["alpha"]
    lr=     HP["lr"]
    weight_decay= HP["wd"]
    bits = HP["bits"]


    model = nn.Sequential(  nn.Linear(4096,256),
                            nn.ReLU(),
                            nn.Linear(256, bits),
                            )
    model.to(device)

    criterion = DTSHLoss()
    optimizer = optim.RMSprop(model.parameters(), lr=lr , weight_decay=weight_decay)

    dataloader = get_dataloader(X_train, y_train, one_hot=do_one_hot)
    historical_lostList = []
    for i in range(1500):
        loss_list = []
        for j,batch  in enumerate(dataloader):
            X_batch = batch[0]
            y_batch = batch[1]

            optimizer.zero_grad()

            u = model(X_batch)
            loss = criterion(u, y_batch.float(), LAMBDA=LAMBDA, ALPHA=ALPHA)
            loss.backward()
            optimizer.step()

            loss_list.append( float(loss) )
        
        
        mean_loss = sum(loss_list) / len(loss_list)
        if i % 10 == 1:
            print(i, mean_loss)
        historical_lostList.append(mean_loss)

        if earlyStop(historical_lostList, n = earlyStop_num): 
            print(i, mean_loss)
            print("Early Stop!!!")
            data["earlyStop"] = True
            break
    
        if time.time() - time_start > timedStop: 
            break


    return model

In [9]:
Nus = [
    {"lambda": 0.5, "alpha": 3, "lr":0.00010, "wd": 0.00001, "bits"   : 24},
    {"lambda": 0.5, "alpha": 3, "lr":0.00010, "wd": 0.00001, "bits"   : 48},
    {"lambda": 0.5, "alpha": 3, "lr":0.00010, "wd": 0.00001, "bits"   : 32},
    {"lambda": 0.5, "alpha": 5, "lr":0.00001, "wd": 0.00001, "bits"   : 12},
       ]

Cifar = [
    {"lambda": 2.0, "alpha": 5, "lr":0.00001, "wd": 0.00001, "bits"   : 48},
    {"lambda": 0.5, "alpha": 5, "lr":0.00010, "wd": 0.00010, "bits"   : 12},
    {"lambda": 1.0, "alpha": 5, "lr":0.00010, "wd": 0.00010, "bits"   : 24},
    {"lambda": 0.5, "alpha": 5, "lr":0.00010, "wd": 0.00010, "bits"   : 32},
       ]

ImgNet = [
    {"lambda": 0.5, "alpha": 5, "lr":0.00010, "wd": 0.00001, "bits"   : 24},
    {"lambda": 0.5, "alpha": 5, "lr":0.00010, "wd": 0.00010, "bits"   : 32},
    {"lambda": 0.5, "alpha": 5, "lr":0.00001, "wd": 0.00010, "bits"   : 48},
    {"lambda": 0.5, "alpha": 5, "lr":0.00001, "wd": 0.00001, "bits"   : 12},
       ]

hp = Nus[0]

X_train = torch.tensor( np.load( root + r"Features/HPO og Validering/CIFAR/X_hpo_Cifar.npy" ) )
y_train = torch.tensor( np.load( root + r"Features/HPO og Validering/CIFAR/y_hpo_CIfar.npy" ) )

model = HPO(hp, X_train, y_train, earlyStop_num=1, timedStop=30) # ! SET EARLY STOP "earlyStop_num" TO 20 (hpo er lavet på 10, så det vil nok også gå hvis vi har travlt)
# ! TIMED STOP ER IS SEKUNDER!!!!

C:\Users\thors\AppData\Local\Temp\ipykernel_26632\4162365391.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.long, device=device)


1 0.9393754923343658


KeyboardInterrupt: 

In [8]:
X_val = torch.tensor( np.load( root + r"Features/HPO og Validering/CIFAR/X_val_Cifar.npy" ) )

model(X_val[22])

tensor([ 0.6974, -0.1630, -1.0359, -0.6633,  0.2369, -0.2210,  0.8083,  0.1142,
        -0.1639,  0.1518,  0.6569,  0.7596, -0.8730,  1.3591,  0.4493, -0.6184,
        -0.4225,  0.2325, -0.6697,  0.9153,  0.1910,  0.6520, -0.6603,  0.8978],
       grad_fn=<ViewBackward0>)